
###지식 그래프(Knowledge Graph) 기반 구독 서비스 정보 복합 추론  엔진

Neo4j 지식 그래프를 기반으로, 사용자의 자연어 질문에 대해 정확하고 풍부한 답변을 제공하는 GraphRAG 시스템을 구현합니다.

단순한 키워드 매칭이나 벡터 검색의 한계를 넘어, 그래프의 구조적 정보(관계)와 벡터의 의미적 정보(유사도)를 결합하여 최적의 검색 결과를 도출합니다.


시스템은 질문의 유형에 따라 세 가지 검색 전략을 통합하여 사용합니다:

1. **[Vector]** Content 노드 기반 벡터 검색 - 내용/혜택 유사도 기반 검색
2. **[VectorCypher]** - 복합 검색
3. **[Text2Cypher]** 자연어를 Cypher로 변환한 구조적 검색



1. 환경 설정 및 DB 연결

In [ ]:
!pip install langchain langchain-community langchain-openai neo4j python-dotenv tiktoken neo4j_graphrag

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: fsspec
    Found exi

In [ ]:
!pip install sentence-transformers

In [ ]:
import os
from neo4j import GraphDatabase, basic_auth
from sentence_transformers import SentenceTransformer
from neo4j_graphrag.embeddings import Embedder
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.retrievers import VectorCypherRetriever
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.indexes import create_vector_index
from neo4j_graphrag.llm import OpenAILLM
from dotenv import load_dotenv
from google.colab import drive
from os import path

drive.mount('/content/drive')

if os.path.exists("/content/drive/MyDrive/project_Demo/start_final/.env.txt"):
  try:
    os.rename("/content/drive/MyDrive/project_Demo/start_final/.env.txt", "/content/drive/MyDrive/project_Demo/start_final/.env")
  except FileNotFoundError:
    print("파일이 존재하지 않습니다.")
    exit(1)
  except OSError as e:
    print(f"파일 이름 변경 중 오류 발생: {e}")
    exit(1)

load_dotenv("/content/drive/MyDrive/project_Demo/start_final/.env", override = True)

NEO4J_URI = "neo4j+s://579bb57f.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PW   = os.getenv("NEO4J_PW")

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=basic_auth(NEO4J_USER, NEO4J_PW)
)

Mounted at /content/drive


2. 임베딩 모델 로드 및 데이터 인덱싱

In [ ]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
EMBED_DIM = 384

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
class LocalEmbedder(Embedder):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text: str) -> list[float]:
        return self.model.encode(text, normalize_embeddings=True).tolist()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()
embedder = LocalEmbedder(MODEL_NAME)

In [ ]:
def enrich_benefit_text(tx):

    q = """

    MATCH (p:Product)-[:HAS_PLAN]->(plan:Plan)-[:OFFERS]->(c:Content {type: 'BENEFIT'})

    SET c.text =  p.name + " " + plan.title + " 혜택: " + c.name + " - " + coalesce(c.detail, '')

    RETURN count(c) as updated_count

    """

    result = tx.run(q)

    return result.single()['updated_count']



with driver.session() as session:

    cnt = session.execute_write(enrich_benefit_text)

    print(f"{cnt}개의 BENEFIT 노드 텍스트가 보강되었습니다.")



with driver.session() as session:

    session.run("MATCH (c:Content {type: 'BENEFIT'}) SET c.embedding = NULL")

    print("BENEFIT 노드 임베딩 초기화 완료. 다시 embed_all_contents()를 실행하세요.")

83개의 BENEFIT 노드 텍스트가 보강되었습니다.
BENEFIT 노드 임베딩 초기화 완료. 다시 embed_all_contents()를 실행하세요.


In [ ]:
def fetch_contents_needing_embedding(tx, limit=200):
    q = """
    MATCH (c:Content)
    WHERE (c.text IS NOT NULL AND c.text <> "")
       OR (c.detail IS NOT NULL AND c.detail <> "")
      AND (c.embedding IS NULL OR c.embedding_model <> $model)
    RETURN c.content_id AS content_id, c.text AS text, c.name AS name, c.detail AS detail
    LIMIT $limit
    """
    return list(tx.run(q, limit=limit, model=MODEL_NAME))

def update_content_embeddings(tx, rows):
    q = """
    UNWIND $rows AS row
    MATCH (c:Content {content_id: row.content_id})
    SET c.embedding = row.embedding,
        c.text = row.text,
        c.embedding_model = $model,
        c.embedding_dim = $dim
    """
    tx.run(q, rows=rows, model=MODEL_NAME, dim=EMBED_DIM)

def embed_all_contents(batch_size=500, max_batches=100):
    print(f"Start Embedding with model: {MODEL_NAME}...")
    total = 0
    with driver.session() as session:
        for i in range(max_batches):
            records = session.execute_read(fetch_contents_needing_embedding, batch_size)
            if not records:
                print("모든 데이터 임베딩 완료.")
                break

            processed_rows = []
            texts_to_embed = []

            for r in records:
                text_content = r["text"]
                if not text_content and r["detail"]:
                    text_content = f"[{r['name']}] {r['detail']}"

                texts_to_embed.append(text_content)
                processed_rows.append({"content_id": r["content_id"], "text": text_content})

            # 벡터 생성
            embeddings = embedder.embed_documents(texts_to_embed)

            rows = []
            for row_data, vec in zip(processed_rows, embeddings):
                row_data["embedding"] = vec
                rows.append(row_data)

            session.execute_write(update_content_embeddings, rows)

            total += len(rows)
            print(f"[Batch {i+1}] {len(rows)}개 저장 (Total: {total})")

embed_all_contents()



Start Embedding with model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...
[Batch 1] 174개 저장 (Total: 174)
[Batch 2] 174개 저장 (Total: 348)
[Batch 3] 174개 저장 (Total: 522)
[Batch 4] 174개 저장 (Total: 696)
[Batch 5] 174개 저장 (Total: 870)
[Batch 6] 174개 저장 (Total: 1044)
[Batch 7] 174개 저장 (Total: 1218)
[Batch 8] 174개 저장 (Total: 1392)
[Batch 9] 174개 저장 (Total: 1566)
[Batch 10] 174개 저장 (Total: 1740)
[Batch 11] 174개 저장 (Total: 1914)
[Batch 12] 174개 저장 (Total: 2088)
[Batch 13] 174개 저장 (Total: 2262)
[Batch 14] 174개 저장 (Total: 2436)
[Batch 15] 174개 저장 (Total: 2610)
[Batch 16] 174개 저장 (Total: 2784)
[Batch 17] 174개 저장 (Total: 2958)
[Batch 18] 174개 저장 (Total: 3132)
[Batch 19] 174개 저장 (Total: 3306)
[Batch 20] 174개 저장 (Total: 3480)
[Batch 21] 174개 저장 (Total: 3654)
[Batch 22] 174개 저장 (Total: 3828)
[Batch 23] 174개 저장 (Total: 4002)
[Batch 24] 174개 저장 (Total: 4176)
[Batch 25] 174개 저장 (Total: 4350)
[Batch 26] 174개 저장 (Total: 4524)
[Batch 27] 174개 저장 (Total: 4698)
[Batch 28] 174개 저장 (Total: 4872)

In [ ]:
### 벡터 인덱스 생성
VECTOR_INDEX_NAME = "vector_index"

try:
    with driver.session() as session:
        session.run(f"DROP INDEX {VECTOR_INDEX_NAME} IF EXISTS")

    create_vector_index(
        driver,
        VECTOR_INDEX_NAME,
        "Content",
        "embedding",
        EMBED_DIM,
        "cosine"
    )
    print(f"[OK] 인덱스({VECTOR_INDEX_NAME}) 생성 완료")
except Exception as e:
    print(f"인덱스 오류: {e}")

[OK] 인덱스(vector_index) 생성 완료


## 검색 전략 1: vector retriever
단순히 키워드가 일치하는 것이 아니라, 문장의 맥락과 의미가 유사한 데이터를 찾는 검색기입니다.
* **용도:**  구체적인 수치보다는 **텍스트의 내용/뉘앙스**를 찾을 때 사용됩니다.
* **작동 원리:** 사용자 질문을 벡터로 변환하여 DB 내 `Content` 노드들과 코사인 유사도(Cosine Similarity)를 계산해 가장 유사한 결과를 반환합니다.

In [ ]:
retriever = VectorRetriever(
        driver=driver,
        index_name=VECTOR_INDEX_NAME,
        embedder=embedder,
        return_properties=["text", "content_id"]
    )

In [ ]:
def check_vector_search(query, top_k=3):
    print(f"질문: {query}\n")

    # 검색 수행
    results = retriever.search(query_text=query, top_k=top_k)

    # 결과 출력
    for i, item in enumerate(results.items):

        content = item.content
        score = item.metadata.get('score', 0.0)

        if isinstance(content, str):
            try:
                import ast
                content = ast.literal_eval(content)
            except:
                pass

        content_id = content.get('content_id', 'Unknown') if isinstance(content, dict) else 'Unknown'
        text_preview = content.get('text', '') if isinstance(content, dict) else str(content)

        print(f"[Rank {i+1}]")
        print(f"  - Score      : {score:.4f}")
        print(f"  - Content ID : {content_id}")
        print(f"  - Text       : {text_preview}")
        print("-" * 50)
check_vector_search(" 티빙 상품에서 크롬캐스트/미러링 제한이 있는 플랜이 있어?", top_k=3)

질문:  티빙 상품에서 크롬캐스트/미러링 제한이 있는 플랜이 있어?

[Rank 1]
  - Score      : 0.7338
  - Content ID : 3f77a86f743bf8f6591ff320e3196de4
  - Text       : 티빙 | KT 프리미엄 혜택: 디바이스 > 크롬캐스트/미러링 - O
--------------------------------------------------
[Rank 2]
  - Score      : 0.7334
  - Content ID : 67d2f80eb24f084aecad1fb63f4f6371
  - Text       : 티빙 | KT 스탠다드 혜택: 디바이스 > 크롬캐스트/미러링 - O
--------------------------------------------------
[Rank 3]
  - Score      : 0.7329
  - Content ID : 8a1440521f7b4a4d1362b57fadf24f0c
  - Text       : 티빙 | KT 베이직 혜택: 디바이스 > 크롬캐스트/미러링 - O
--------------------------------------------------


In [ ]:
check_vector_search("커피 기프티쇼는 언제 발송되고 유효기간은 얼마야?", top_k=3)

질문: 커피 기프티쇼는 언제 발송되고 유효기간은 얼마야?

[Rank 1]
  - Score      : 0.7995
  - Content ID : KT_1579_policy_5
  - Text       : 련해서는 아래의 정책이 적용됩니다. 1) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이내 해지하시는 경우에는, 티빙 콘텐츠와 스타벅스 기프티쇼 이용 여부에 따라 해지 처리됩니다. 가입 후 7일 이내 티빙 콘텐츠 및 스타벅스 기프티쇼 사용 이력이 없는 경우, 즉시 해지가 가능하며, 명세서에 청구되지 않습니다. 가입 후 7일 이내 티빙 콘텐츠 또는 스타벅스 기프티쇼 사용 이력이 존재하는 경우, 즉시 해지 불가 및 해지 예약되며 이용요금이 부과됩니다. 예시) 최초 가입일이 1월 3일인 경우, 서비
--------------------------------------------------
[Rank 2]
  - Score      : 0.7765
  - Content ID : KT_1579_policy_6
  - Text       : 가 및 해지 예약되며 이용요금이 부과됩니다. 예시) 최초 가입일이 1월 3일인 경우, 서비스 이용료 결제일은 매월 3일이며 매월 3일 ~ 9일 동안 콘텐츠 및 스타벅스 기프티쇼 사용 이력이 없는 경우 해지 시 이용료가 부과되지 않음 2) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이후에 해지하시는 경우에는, 즉시 해지 불가 및 해지 예약되고 이용요금은 해당월의 이용료가 전액 청구되며, 티빙 콘텐츠와 스타벅스 기프티쇼를 1달 동안 지속 이용 가능합니다. 예시) 최초 가입일이 1월 3일인 경우,
--------------------------------------------------
[Rank 3]
  - Score      : 0.7746
  - Content ID : KT_1599_policy_5
  - Text       : le 계정 등록을 진

In [ ]:
check_vector_search("티빙 광고형 스탠다드+메가MGC커피 요금제에서 동시 시청 가능 수 알려줘", top_k=3)

질문: 티빙 광고형 스탠다드+메가MGC커피 요금제에서 동시 시청 가능 수 알려줘

[Rank 1]
  - Score      : 0.8222
  - Content ID : 15e10c1bf6bea07cb1f24b5f88651925
  - Text       : 티빙+메가MGC커피 | KT 티빙 광고형 스탠다드+메가MGC커피 혜택: 티빙 > 동시 시청 가능 수/ 멀티 프로필 - 2 / 4
--------------------------------------------------
[Rank 2]
  - Score      : 0.8160
  - Content ID : d4c9497b54d6b11122830540e9288d6e
  - Text       : 티빙+메가MGC커피 | KT 티빙 베이직+메가MGC커피 혜택: 티빙 > 동시 시청 가능 수/ 멀티 프로필 - 1 / 4
--------------------------------------------------
[Rank 3]
  - Score      : 0.8135
  - Content ID : 1c9aa68ef703b9db61a5c91eb2eaae41
  - Text       : 티빙+메가MGC커피 | KT 티빙 스탠다드+메가MGC커피 혜택: 티빙 > 동시 시청 가능 수/ 멀티 프로필 - 2 / 4
--------------------------------------------------


In [ ]:
check_vector_search("티빙+스타벅스 요금제의 가입 불가 고객의 기준에 대해서 알려줘", top_k=3)

질문: 티빙+스타벅스 요금제의 가입 불가 고객의 기준에 대해서 알려줘

[Rank 1]
  - Score      : 0.8072
  - Content ID : KT_1579_policy_9
  - Text       : 시 이용료가 부과되지 않음 가입 후 7일 이내 티빙 콘텐츠 및 스타벅스 기프티쇼 사용 이력이 없는 경우, 즉시 해지가 가능하며, 명세서에 청구되지 않습니다. 가입 후 7일 이내 티빙 콘텐츠 또는 스타벅스 기프티쇼 사용 이력이 존재하는 경우, 즉시 해지 불가 및 해지 예약되며 이용요금이 부과됩니다. 2) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이후에 해지하시는 경우에는, 즉시 해지 불가 및 해지 예약되고 이용요금은 해당월의 이용료가 전액 청구되며, 티빙 콘텐츠와 스타벅스 기프티쇼를 1달 동
--------------------------------------------------
[Rank 2]
  - Score      : 0.8064
  - Content ID : KT_1665_policy_11
  - Text       : 료 결제일인 3일 이전까지 해지 시 '월 이용료 전액 부과'되며 익월 2일까지 티빙 콘텐츠와 메가MGC커피 기프티쇼 이용 가능 이미 유료 이용 중인 티빙 계정이 존재할 경우 티빙 이용권이 중복으로 등록되어 이중 과금 될 수 있습니다. 계정 등록 시 기존 유료 이용 중인 티빙 이용권은 '티빙 > 고객센터 > 다음 회차 결제 해지' 메뉴에서 해지 신청 또는 티빙 고객센터(1670-1525)로 문의 주시기 바랍니다. 유료 이용권 잔여기간에 대한 환불을 원하시는 경우는 '티빙 > 고객센터 > 1:1 문의' 또는 'tving@cj.net'으
--------------------------------------------------
[Rank 3]
  - Score      : 0.7987
  - Content ID : KT_1579_policy_5
  - Text       : 련해서

In [ ]:
check_vector_search("회선이 일시정지되면 부가서비스는 어떻게 돼?", top_k=3)

질문: 회선이 일시정지되면 부가서비스는 어떻게 돼?

[Rank 1]
  - Score      : 0.8238
  - Content ID : KT_1667_policy_4
  - Text       : 서비스를 가입한 모바일 회선이 일시중지/정지 상태이면, 가입한 부가서비스는 해지 예약 상태로 자동 변경되며 다음 결제일 도래전까지 서비스 이용 가능합니다. 다음 결제일 전에 일시중지/정지가 해제될 경우 가입한 부가서비스는 해지되지 않고 계속해서 이용 가능합니다. 부가서비스를 가입한 모바일 회선이 해지될 경우 가입한 부가서비스는 자동 해지됩니다. 제휴사의 사정에 의해 서비스 내용이나 상시 할인 혜택은 변경, 또는 종료될 수 있습니다. 부가서비스 가입 후, 고객님께 발송되는 계정등록 안내문자 URL 또는 마이케이티(마이>요금/서비스>부...
--------------------------------------------------
[Rank 2]
  - Score      : 0.8238
  - Content ID : KT_1610_policy_4
  - Text       : 서비스를 가입한 모바일 회선이 일시중지/정지 상태이면, 가입한 부가서비스는 해지 예약 상태로 자동 변경되며 다음 결제일 도래전까지 서비스 이용 가능합니다. 다음 결제일 전에 일시중지/정지가 해제될 경우 가입한 부가서비스는 해지되지 않고 계속해서 이용 가능합니다. 부가서비스를 가입한 모바일 회선이 해지될 경우 가입한 부가서비스는 자동 해지됩니다. 제휴사의 사정에 의해 서비스 내용이나 상시 할인 혜택은 변경, 또는 종료될 수 있습니다. 부가서비스 가입 후, 고객님께 발송되는 계정등록 안내문자 URL 또는 마이케이티(마이>요금/서비스>부...
--------------------------------------------------
[Rank 3]
  - Score      : 0.8103
  - Content ID : KT_1668_policy_4
  - Text       : 습니다. 부가서

## 검색 전략 2: VectorCypher Retriever

벡터 검색과 그래프 탐색(Graph Traversal)을 결합한 방식입니다.
단순히 "커피 제공"이라는 텍스트만 찾는 것이 아니라, 그 혜택이 "어떤 상품"의 "얼마짜리 요금제"에 포함되어 있는지 역추적하여 정보를 결합합니다.

* **Cypher Query 로직:**
    1. **Vector Search:** 질문과 유사한 `Content`(혜택/약관) 노드를 먼저 찾습니다.
    2. **Graph Traversal:** 찾은 노드에서 `(:Product)-[:HAS_PLAN]->(:Plan)-[:OFFERS]->(:Content)` 관계를 역으로 거슬러 올라갑니다.
    3. **Aggregation:** 연결된 `Product` 이름, `Plan` 가격 정보를 함께 수집(Collect)하여 LLM에게 풍부한 맥락(Rich Context)을 제공합니다.

In [ ]:
# [시나리오]
# 1. 벡터 검색(Vector Search)으로 질문과 의미가 가장 유사한 'Content(약관/정책)' 또는 'Benefit(혜택 상세)' 노드를 찾음.
# 2. 찾은 노드가 '약관(Policy)'이라면: 해당 약관을 보유한 'Product(상품)'를 역추적.
# 3. 찾은 노드가 '혜택(Benefit)'이라면: 해당 혜택을 제공하는 'Plan(요금제)'을 거쳐 상위 'Product(상품)'를 역추적.
# 4. 최종적으로 식별된 'Product'를 기준으로, 포함된 'Service(서비스)' 목록과 제공하는 모든 'Plan(요금제)'의 가격 정보를 함께 수집하여 반환.

retrieval_query1 = """
// Vector search 결과
WITH node AS content, score

CALL (content) {
  OPTIONAL MATCH (p:Product)-[:HAS_POLICY]->(content)
  RETURN collect(DISTINCT p) AS policyProducts
}

CALL (content) {
  OPTIONAL MATCH (p:Product)-[:HAS_PLAN]->(:Plan)-[:OFFERS]->(content)
  RETURN collect(DISTINCT p) AS benefitProducts
}

WITH content, score,
     CASE
       WHEN content.type = 'POLICY'  THEN policyProducts
       WHEN content.type = 'BENEFIT' THEN benefitProducts
       ELSE (policyProducts + benefitProducts)
     END AS products

UNWIND products AS product
WITH DISTINCT content, score, product
WHERE product IS NOT NULL

CALL (product) {
  OPTIONAL MATCH (product)-[:INCLUDES_SERVICE]->(s:Service)
  RETURN collect(DISTINCT {
    service_id: s.service_id,
    name: s.name,
    category: s.category
  }) AS services
}

CALL (product) {
  OPTIONAL MATCH (product)-[:HAS_PLAN]->(pl:Plan)
  OPTIONAL MATCH (pl)-[:BASED_ON]->(bp:BasePlan)
  RETURN collect(DISTINCT {
    plan_id: pl.plan_id,
    plan_name: pl.title,
    base_plan_key: bp.key,
    price_regular: pl.price_regular,
    price_kt: pl.price_kt,
    price_promo: pl.price_promo,
    currency: pl.currency,
    billing_period: pl.billing_period
  }) AS plans
}

OPTIONAL MATCH (prov:Provider)-[:PROVIDES]->(product)

RETURN
  content.content_id AS content_id,
  content.type AS content_type,
  content.text AS matched_text,
  product.product_id AS product_id,
  product.name AS product_name,
  product.url AS product_url,
  prov.name AS provider_name,
  services AS included_services,
  plans AS related_plans,
  score
ORDER BY score DESC
LIMIT 5
"""

vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name="vector_index",
    retrieval_query=retrieval_query1,
    embedder=embedder
)

In [ ]:
# 테스트 코드: Agent 없이 Retriever가 잘 작동하는지 확인
test_query = "티빙 베이직+메가MGC커피 가격은 얼마야?"
results = vector_cypher_retriever.search(query_text=test_query, top_k=3)

print(f"검색된 항목 수: {len(results.items)}")
for item in results.items:
    # item.content는 위 쿼리의 RETURN 값들을 담고 있습니다.
    print(item.content)

검색된 항목 수: 3
<Record content_id='KT_1665_policy_7' content_type='POLICY' matched_text=" 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이후에 해지하시는 경우에는, 즉시 해지 불가 및 해지 예약되고 이용요금은 해당월의 이용료가 전액 청구되며, 티빙 콘텐츠와 메가MGC커피 기프티쇼를 1달 동안 지속 이용 가능합니다. 예시) 최초 가입일이 1월 3일인 경우, 서비스 이용료 결제일은 매월 3일이며 매월 10일부터 서비스 이용료 결제일인 3일 이전까지 해지 시 '월 이용료 전액 부과'되며 익월 2일까지 티빙 콘텐츠와 메가MGC커피 기프티쇼 이용 가능 1) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시" product_id='KT_1665' product_name='티빙+메가MGC커피 | KT' product_url='https://m.product.kt.com/static/prodetail/1665/mobile/detail_view/m_ott_pop_tving_mgc_coffee.html' provider_name='KT' included_services=[{'service_id': 'OTT_TVING', 'name': 'TVING', 'category': 'OTT'}, {'service_id': 'COFFEE_MGC', 'name': '메가 MGC', 'category': 'COFFEE'}] related_plans=[{'billing_period': 'monthly', 'price_regular': 22100, 'price_promo': 17000, 'price_kt': 20100, 'currency': 'KRW', 'plan_name': '티빙 프리미엄+메가MGC커피', 'plan_id': 'KT_1665_TVING_PREMIUM_MGC', 'base_plan_key': 'tving_premium'}, {'billing_period': 'monthly', 'price_re

## 검색 전략 3: Text2Cypher Retriever

LLM에게 그래프 스키마(Schema)를 제공하여, 사용자의 자연어 질문을 실행 가능한 **Cypher Query**로 변환해 DB를 직접 조회합니다.
.

In [ ]:
def get_schema():
    with driver.session() as session:
        node_props = session.run("""
            CALL db.schema.nodeTypeProperties()
            YIELD nodeType, propertyName, propertyTypes
            WHERE propertyName IS NOT NULL
            RETURN nodeType, collect(propertyName) as properties
        """).data()

        rels = session.run("""
             MATCH (n)-[r]->(m)
             RETURN DISTINCT labels(n)[0] as source, type(r) as rel_type, labels(m)[0] as target
             LIMIT 50
         """).data()

        schema_text = "=== Graph Schema ===\n"

        schema_text += "[Nodes]\n"
        for n in node_props:
            schema_text += f"- {n['nodeType']} {n['properties']}\n"

        schema_text += "\n[Relationships]\n"
        for r in rels:
            schema_text += f"(:{r['source']})-[:{r['rel_type']}]->(:{r['target']})\n"

        return schema_text

# 스키마 확인
neo4j_schema = get_schema()
print(neo4j_schema)

=== Graph Schema ===
[Nodes]
- :`Product` ['product_id', 'name', 'url', 'fetched_at', 'promotion_info']
- :`Service` ['service_id', 'name', 'category']
- :`Plan` ['plan_id', 'title', 'price_regular', 'price_kt', 'price_promo', 'currency', 'billing_period']
- :`Content` ['content_id', 'name', 'detail', 'type', 'text', 'index', 'embedding', 'embedding_model', 'embedding_dim']
- :`BasePlan` ['key']
- :`Provider` ['provider_id', 'name']

[Relationships]
(:Provider)-[:PROVIDES]->(:Product)
(:Product)-[:INCLUDES_SERVICE]->(:Service)
(:Product)-[:HAS_PLAN]->(:Plan)
(:Product)-[:HAS_POLICY]->(:Content)
(:Plan)-[:BASED_ON]->(:BasePlan)
(:Plan)-[:OFFERS]->(:Content)



In [ ]:
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError(f"API Key를 찾을 수 없습니다. 파일을 확인해주세요.")


os.environ["OPENAI_API_KEY"] = google_api_key
os.environ["OPENAI_BASE_URL"] = "https://generativelanguage.googleapis.com/v1beta/openai/"
llm = OpenAILLM(
    model_name="gemini-2.0-flash-exp", #llama-3.3-70b-versatile 무료 사용량 초과로 대체 모델 사용
    model_params={
        "temperature": 0,
        "max_tokens": 2000
    }
)

print(f" LLM 설정 완료: {llm.model_name}")

 LLM 설정 완료: gemini-2.0-flash-exp


In [ ]:
examples = [
"""
USER INPUT: 티빙 베이직+메가MGC커피 포함된 상품 정보 알려줘
CYPHER QUERY:
MATCH (p:Product)-[:HAS_PLAN]->(pl:Plan)
WHERE pl.title CONTAINS "티빙 베이직+메가MGC커피"
OPTIONAL MATCH (p)-[:INCLUDES_SERVICE]->(s:Service)
RETURN p.name, p.url,
       collect(DISTINCT s.name) AS services,
       pl.title, pl.price_regular, pl.price_kt, pl.price_promo, pl.billing_period;
""",

"""
USER INPUT: 티빙 베이직+메가MGC커피 가격은 얼마야? KT 할인가도 알려줘
CYPHER QUERY:
MATCH (p:Product)-[:HAS_PLAN]->(pl:Plan)
WHERE p.name CONTAINS '티빙' AND p.name CONTAINS '메가' AND pl.title CONTAINS '베이직'
RETURN p.name AS product_name, pl.title AS plan_title,
       pl.price_regular, pl.price_kt, pl.price_promo, pl.currency, pl.billing_period
LIMIT 5
""",

"""
USER INPUT: 메가커피랑 같이 주는 상품들 목록 보여줘
CYPHER QUERY:
MATCH (p:Product)-[:INCLUDES_SERVICE]->(s:Service)
WHERE s.category = 'COFFEE' AND p.name CONTAINS '메가'
RETURN p.product_id, p.name, collect(DISTINCT s.name) AS services, p.url
ORDER BY p.name
LIMIT 20
""",

"""
USER INPUT: 티빙+메가MGC커피 상품에 포함된 서비스가 뭐야?
CYPHER QUERY:
MATCH (p:Product)-[:INCLUDES_SERVICE]->(s:Service)
WHERE p.name CONTAINS '티빙' AND p.name CONTAINS '메가'
RETURN p.name AS product_name, collect(DISTINCT {service_id:s.service_id, name:s.name, category:s.category}) AS included_services
LIMIT 5
""",

"""
USER INPUT: 티빙+메가MGC커피 환불 관련 약관 내용 보여줘
CYPHER QUERY:
MATCH (p:Product)-[:HAS_POLICY]->(c:Content)
WHERE p.name CONTAINS '티빙' AND p.name CONTAINS '메가' AND c.text CONTAINS "환불"
RETURN p.name AS product_name, c.content_id, c.type, c.text
LIMIT 3
""",

"""
USER INPUT: 광고형 스탠다드 요금제 혜택(화질/동시접속/광고/미러링)을 알려줘
CYPHER QUERY:
MATCH (p:Product)-[:HAS_PLAN]->(pl:Plan)-[:OFFERS]->(c:Content)
WHERE pl.title CONTAINS '광고형' AND pl.title CONTAINS '스탠다드'
RETURN p.name AS product_name, pl.title AS plan_title, c.content_id, c.type, c.text
ORDER BY p.name
LIMIT 10
""",

"""
USER INPUT: 크롬캐스트/미러링이 불가한 요금제(또는 상품) 찾아줘
CYPHER QUERY:
MATCH (c:Content)
WHERE c.text CONTAINS '크롬캐스트' OR c.text CONTAINS '미러링'
WITH c
MATCH (p:Product)-[:HAS_POLICY]->(c)
RETURN
  p.name AS product_name,
  c.content_id AS content_id,
  c.text AS content_text
LIMIT 5

UNION

MATCH (c2:Content)
WHERE c2.text CONTAINS '크롬캐스트' OR c2.text CONTAINS '미러링'
WITH c2
MATCH (p2:Product)-[:HAS_PLAN]->(:Plan)-[:OFFERS]->(c2)
RETURN
  p2.name AS product_name,
  c2.content_id AS content_id,
  c2.text AS content_text
LIMIT 5

""",

"""
USER INPUT: 알뜰폰(MVNO)도 가입 가능한지 약관 근거로 알려줘
CYPHER QUERY:
MATCH (p:Product)-[:HAS_POLICY]->(c:Content)
WHERE c.text CONTAINS '알뜰폰' OR c.text CONTAINS 'MVNO'
RETURN p.name AS product_name, c.content_id, c.text
ORDER BY p.name
LIMIT 10
""",

"""
USER INPUT: 디즈니플러스+스타벅스 상품의 프리미엄 플랜 가격과 혜택 요약해줘
CYPHER QUERY:
MATCH (p:Product)-[:HAS_PLAN]->(pl:Plan)
WHERE p.name CONTAINS '디즈니' AND p.name CONTAINS '스타벅스' AND pl.title CONTAINS '프리미엄'
OPTIONAL MATCH (pl)-[:OFFERS]->(c:Content)
RETURN p.name AS product_name, pl.title AS plan_title,
       pl.price_regular, pl.price_kt, pl.price_promo, pl.currency, pl.billing_period,
       collect(DISTINCT {content_id:c.content_id, type:c.type, text:c.text}) AS benefit_contents
LIMIT 5
""",

"""
USER INPUT: 디즈니플러스+스타벅스 요금제에서 스타벅스 기프티쇼 유효기간이 며칠이야?
CYPHER QUERY:
MATCH (p:Product)-[:HAS_POLICY]->(c:Content)
WHERE  p.name CONTAINS '디즈니'  AND (p.name CONTAINS '스타벅스' OR c.text CONTAINS '스타벅스')
  AND (c.text CONTAINS '유효기간' OR c.text CONTAINS '30일')
RETURN p.name AS product_name, c.content_id, c.text
LIMIT 5
""",

"""
USER INPUT: KT 구독 상품 중 가장 비싼 요금제는 뭐야?
CYPHER QUERY:
MATCH (prov:Provider)-[:PROVIDES]->(p:Product)-[:HAS_PLAN]->(pl:Plan)
RETURN prov.name AS provider, p.name AS product_name, pl.title AS plan_title,
       pl.price_regular, pl.price_kt, pl.price_promo
ORDER BY pl.price_regular DESC
LIMIT 1
""",

"""
USER INPUT: 메가MGC커피 포함 상품 중 월 2만원 이하 플랜 있어?
CYPHER QUERY:
MATCH (p:Product)-[:INCLUDES_SERVICE]->(s:Service)
WHERE p.name CONTAINS '메가' OR s.name CONTAINS '메가'
WITH DISTINCT p
MATCH (p)-[:HAS_PLAN]->(pl:Plan)
WHERE pl.price_regular <= 20000
RETURN p.name AS product_name, pl.title AS plan_title,
       pl.price_regular, pl.price_kt, pl.price_promo
ORDER BY pl.price_regular ASC
LIMIT 20
"""
]

text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=neo4j_schema,
    examples=examples,
)

## 3가지 Retriever(Vector, VectorCypher, Text2Cypher)를 하나로 통합하여 ToolsRetriever와 GraphRAG를 구축
사용자 질문의 특성에 따라 서로 다른 검색 전략을 선택적으로 적용하기 위해 총 세 가지 Retriever(Vector, VectorCypher, Text2Cypher)를 설계하였습니다.

질문 라우팅(Question Routing) 단계를 검색 이전에 명시적으로 도입하여 사용자 질문을 사전에 분석하여 질문 유형을 분류 후 분류 결과에 따라 가장 적합한  retriever 선택하고, 검색이 필요한 경우에만 tool retriever를 사용합니다.

In [ ]:
from neo4j_graphrag.retrievers import ToolsRetriever

# (1) Vector Retriever Tool: 단순 의미 검색
# - 상황: 사용자가  정책이나 약관 내용을 찾을 때
vector_tool = retriever.convert_to_tool(
    name="Policy_Vector_Search",
    description="사용자가 '해지', '환불', '유의사항', '혜택', '화질', '스펙' 등 약관이나 혜택 정보, 정책의 뉘앙스를 물어볼 때 사용합니다. 구체적인 수치보다는 텍스트의 의미를 찾을 때 적합합니다."
)

# (2) VectorCypher Retriever Tool: 문맥 + 구조적 정보(상품/요금제) 결합
# - 상황: 약관을 물어보면서 동시에 그 약관이 적용되는 상품명이나 요금제 리스트를 같이 알아야 할 때
vector_cypher_tool = vector_cypher_retriever.convert_to_tool(
    name="Comprehensive_Product_Search",
    description=( "약관/혜택 텍스트를 찾고, 그 텍스트가 적용되는 상품(Product)과 요금제(Plan) 목록(가격 포함)을 함께 반환할 때 사용. ","사용자가 약관 내용과 함께, 해당 약관이 포함된 '상품 정보'와 '요금제 목록(가격 포함)'을 한꺼번에 알고 싶을 때 가장 유용합니다. (예: '커피 주는 요금제 설명해줘')" )
)

# (3) Text2Cypher Retriever Tool: 정확한 사실/화질 등)을 콕 집어서 물어볼 때
text2cypher_tool = text2cypher_retriever.convert_to_tool(
    name="Factual_Data_Search",
    description="'가격', '요금제 이름', '혜택' 등 DB에 저장된 구체적인 사실이나 수치를 검색할 때 사용합니다. (예: '프리미엄 요금제 얼마야?')"
)
tools_retriever = ToolsRetriever(
    driver=driver,
    llm=llm,
    tools=[vector_tool, vector_cypher_tool, text2cypher_tool]
)

print(" ToolsRetriever 설정 완료")

 ToolsRetriever 설정 완료


In [ ]:
from neo4j_graphrag.generation import GraphRAG, RagTemplate
prompt_template = RagTemplate(
    template="""당신은 구독 상품 정보를 제공하는 전문 상담 AI입니다.

질문: {query_text}

검색된 정보(Context):
{context}

지침:
1. 제공된 검색 결과(Context)가 JSON 형식이거나 리스트일 수 있습니다. 내용을 잘 파석하여 답변하세요.
2. 사용자의 질문 유형에 맞춰 답하세요:
   - 정책/환불/해지/제한: 핵심 규정 요약 + 근거(가능하면 문장 일부) + 적용 상품(있으면)
   - 가격/요금제 비교: 요금제명 + (정가/KT가/프로모션가) + 과금주기
   - 혜택/스펙: 혜택 요약 + 해당 플랜/상품
3. 약관이나 주의사항이 있다면 핵심 내용을 요약해서 전달하세요.
4. 검색 결과에 없는 내용은 "죄송하지만 해당 정보는 찾을 수 없습니다"라고 말하고 추측하지 마세요.
5. 금액은 '원' 단위로 명확히 표기하세요.

답변 형식 예시:
- [상품명] (상품 URL)
  주요 요금제: [요금제 이름] - [가격]원
  관련 정책/내용: [내용 요약]

답변:""",
    expected_inputs=["context", "query_text"]
)

graphrag = GraphRAG(
    llm=llm,
    retriever=tools_retriever,
    prompt_template=prompt_template
)

print(" GraphRAG(Agent) 준비 완료")

 GraphRAG(Agent) 준비 완료


In [ ]:

graphrag_vector = GraphRAG(
    llm=llm,
    retriever=retriever,  # VectorRetriever (의미 유사 텍스트)
    prompt_template=prompt_template
)

graphrag_vectorcypher = GraphRAG(
    llm=llm,
    retriever=vector_cypher_retriever,  # VectorCypherRetriever (텍스트+상품/요금제 역추적)
    prompt_template=prompt_template
)

graphrag_text2cypher = GraphRAG(
    llm=llm,
    retriever=text2cypher_retriever,  # Text2CypherRetriever (가격/사실)
    prompt_template=prompt_template
)

# ToolsRetriever는 “정말 애매할 때만” 쓰는 최후수단
graphrag_tools = GraphRAG(
    llm=llm,
    retriever=tools_retriever,
    prompt_template=prompt_template
)

print("[OK] Routed GraphRAGs ready")


[OK] Routed GraphRAGs ready


In [ ]:
import re
def route_query(q: str) -> str:
    q = q.strip()

    # (1) COMPREHENSIVE: 구조 + 복합 요구
    if re.search(
        r"(같은\s*base|기준으로|번들|단품|포함한|관련\s*상품|"
        r"혜택.*가격|가격.*혜택|혜택.*요금|요금.*혜택|"
        r"차이.*혜택|혜택.*차이|"
        r"여러|목록|비교해서|정리해줘)",
        q
    ):
        return "COMPREHENSIVE"

    # (2) FACT: 단일 수치/정확한 값
    if re.search(
        r"(얼마야|얼마인지|가격이\s*뭐|요금이\s*뭐|가격|할인|요금|얼마|정가|"
        r"정가|월\s*\d+|원\b)",
        q
    ):
        return "FACT"

    # (3) POLICY
    if re.search(r"(약관|정책|환불|해지|청구|위약금|무료체험)", q):
        return "POLICY"

    return "TOOLS"


In [ ]:
# GraphRAG 대화형 인터페이스

print(" 'exit', 'quit', 'q'를 입력하면 종료됩니다.")

while True:
    user_query = input("\n[질문 입력]: ").strip()

    if user_query.lower() in ['exit', 'quit', 'q']:
        print("\n챗봇을 종료합니다.")
        break

    # 빈 입력 방지
    if not user_query:
        continue

    print("\n 검색 및 답변 생성 중...")

    try:
        mode = route_query(user_query)

        if mode == "FACT":
            result = graphrag_text2cypher.search(query_text=user_query, return_context=True)

        elif mode == "COMPREHENSIVE":
            result = graphrag_vectorcypher.search(query_text=user_query, return_context=True)

        elif mode == "POLICY":
            result = graphrag_vector.search(query_text=user_query, return_context=True)

        else:  # "TOOLS"
            result = graphrag_tools.search(query_text=user_query, return_context=True)

        print(f" [라우팅]: {mode}")
        print(f" [질문]: {user_query}")
        print("\n" + "="*50)
        print("[답변]")
        print(result.answer)
        print("="*50)

    except Exception as e:
        print(f"\n 오류가 발생했습니다: {e}")

 'exit', 'quit', 'q'를 입력하면 종료됩니다.

[질문 입력]:  KT + 티빙 프리미엄 가격은 얼마고 화질은 어때?

 검색 및 답변 생성 중...
 [라우팅]: FACT
 [질문]: KT + 티빙 프리미엄 가격은 얼마고 화질은 어때?

[답변]
KT 티빙 프리미엄 요금제 정보입니다.

*   **티빙 프리미엄 + 메가MGC커피:**
    *   정가: 22,100원
    *   KT가: 20,100원
    *   프로모션가: 17,000원
    *   최대 화질: 1080p FHD + 4K (일부 콘텐츠)
*   **티빙 프리미엄 + 스타벅스:**
    *   정가: 21,700원
    *   KT가: 19,700원
    *   프로모션가: 17,000원
    *   최대 화질: 1080p FHD + 4K (일부 콘텐츠)
*   **티빙 프리미엄 (단독):**
    *   정가/KT가: 17,000원
    *   프로모션가: 16,000원
    *   최대 화질: 1080p FHD + 4K

모든 요금제는 월 단위로 과금됩니다.

[질문 입력]: 알뜰폰 요금제 쓰고 있는데 kt + 티빙 구독 상품 가입할 수 있어?

 검색 및 답변 생성 중...
 [라우팅]: FACT
 [질문]: 알뜰폰 요금제 쓰고 있는데 kt + 티빙 구독 상품 가입할 수 있어?

[답변]
알뜰폰(MVNO) 이용 고객은 "티빙+메가MGC커피 | KT" 또는 "티빙+스타벅스 | KT" 구독 서비스에 가입할 수 없습니다. 해당 서비스는 5G/LTE 요금제 이용 중인 KT 고객에게만 할인 혜택이 제공됩니다.


[질문 입력]: KT 구독 상품 중 가장 비싼 요금제는 뭐야?

 검색 및 답변 생성 중...
 [라우팅]: FACT
 [질문]: KT 구독 상품 중 가장 비싼 요금제는 뭐야?

[답변]
KT 구독 상품 중 가장 비싼 요금제는 다음과 같습니다.

*   **티빙 프리미엄+메가MGC커피**: 정가 22,100원, KT가 20,100원, 프로모션가 17,

In [ ]:
driver.close()